Importation des librairies utiles

In [39]:
import requests
import pandas as pd

Création du dictionnaire des Localisation avec leur URL respectives

In [40]:
#Localisation des données
dict_association_localisation = {'AGGLOMÉRATION LE COTENTIN':'https://vigilo.velorution-cherbourg.fr/get_issues.php?',
                                 'AGGLOMÉRATION TROYENNE':'https://vigilo.troyesenselle.fr/get_issues.php?', 
                                 'AIX MARSEILLE PROVENCE METROPOLE':'https://vigilo.velosenville.org/get_issues.php?',
                                 'BASSIN D\'ARCACHON':'https://vigilo.velobassin.org/get_issues.php?',
                                 'BEAUVAISIS':'https://vigilo.vellovaque.fr/get_issues.php?',
                                 'BORDEAUX MÉTROPOLE':'https://vigilo.cycles-manivelles.org/get_issues.php?',
                                 'BREST MÉTROPOLE':'https://vigilo.bapav.org/get_issues.php?',
                                 'EVREUX':'https://vigilo.cube27.org/get_issues.php?',
                                 'GRAND MONTPELLIER':'https://vigilo.jesuisundesdeux.org/get_issues.php?',
                                 'LA ROCHELLE':'https://vigilo.-lr.corolo.fr/get_issues.php?',
                                 'LORIENT AGGLO':'https://vigilo.claav.fr/get_issues.php?',
                                 'MONS':'https://vigilo.geckonline.be/get_issues.php?',
                                 'NANCY AGGLOMÉRATION':'https://vigilo.rigelk.eu/get_issues.php?',
                                 'NANTES MÉTROPOLE':'https://vigilo.placeauvelo-nantes.fr/get_issues.php?',
                                 'NORD-ISÈRE (APIE)':'https://vigilo.apie-asso.net/get_issues.php?',
                                 'PARIS SACLAY':'https://api.vigilo-parissaclay.mdb-idf.org/get_issues.php?',
                                 'PAYS DE CHÂTEAUGIRON':'https://pouliquen.net/vigilo/get_issues.php?',
                                 'PÉRIGUEUX AGGLO':'https://vigilo.velorution-perigourdine.fr/get_issues.php?',
                                 'SAINT MARCELLIN VERCORS ISÈRE COMMUNAUTÉ':'https://vigilo.roulavelo.org/get_issues.php?',
                                 'SAINT-BRIEUC ARMOR AGGLOMÉRATION':'https://api-vigilo.velo-utile.fr/get_issues.php?',
                                 'STRASBOURG':'https://www.strasbourgavelo.eu/get_issues.php?',
                                 'SUD OISE':'https://vigilo.au5v.fr//get_issues.php?',
                                 'THAU':'https://prod-sete.vigilo.velocite-montpellier.fr/get_issues.php?',
                                 'VANNES':'https://vigilo.kaz.bzh/get_issues.php?'
                                }

df_localisation = pd.DataFrame(dict_association_localisation.keys(), columns=["Localisation"])
df_localisation.set_index("Localisation", inplace=True)
df_localisation

""
Localisation
AGGLOMÉRATION LE COTENTIN
AGGLOMÉRATION TROYENNE
AIX MARSEILLE PROVENCE METROPOLE
BASSIN D'ARCACHON
BEAUVAISIS
BORDEAUX MÉTROPOLE
BREST MÉTROPOLE
EVREUX
GRAND MONTPELLIER


Création du dictionnaire avec les différentes catégories de signalements et leur numéro

In [41]:
# URL de l'API pour récupérer la liste des catégories
url_categories = 'https://vigilo-bf7f2.firebaseio.com/categorieslist.json'
resp = requests.get(url_categories)
resp.raise_for_status()

# Convertir la liste de catégories en un dictionnaire avec le nom de la catégorie comme clé et l'identifiant de catégorie comme valeur
categories_dict = {cat["catname"]: str(cat["catid"]) for cat in resp.json()}

# Créer un dataframe à partir du dictionnaire
df_categories = pd.DataFrame(categories_dict.items(), columns=["Catégories", "id"])

# Changer l'index et trier par id
df_categories = df_categories.set_index("id")

# Afficher le dataframe
df_categories


,Catégories
id,
8,Absence d'aménagement
3,Aménagement mal conçu
4,Défaut d'entretien
5,Absence d'arceaux de stationnement
2,Véhicule ou objet gênant
9,"Accident, chute, incident"
6,"Signalisation, marquage"
7,Incivilité récurrente sur la route
10,Vol ou dégradation de vélo


Initialisation des paramètres

In [42]:
# Paramètres de recherche

# Filtre de recherche par localisation, exemple : localisation = "BREST MÉTROPOLE"
location_filter = "GRAND MONTPELLIER"

# Filtre des catégories à afficher, exemple : category_filter = "Absence d'aménagement"
# donne la catégorie Absence d'aménagement
category_filter = None

Création du dataframe des données

In [43]:
def data_collection():
    try:
        resp = requests.get(dict_association_localisation[location_filter])
        resp.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
        df = pd.DataFrame(resp.json())
        return df[categories_dict[category_filter]] if category_filter else df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        return None
df = data_collection()

test

In [44]:
df

,token,coordinates_lat,coordinates_lon,address,comment,explanation,time,status,group,categorie,approved,cityname
0,E3B79D4F,43.66135125488785,3.9750599116086964,"Avenue Mendès France, Vendargues",égoïsme,,1711214460,0,0,2,0,Vendargues
1,FA7BDFC7,43.65886846360422,3.972467556595802,"Rue du Stade, Vendargues",encore le même GCUM #coachfootball,,1711128120,0,0,2,0,Vendargues
2,8F0414E1,43.655721167076656,3.917360901832581,"Impasse Le Ribet, Jacou",espacement trop serré pour vélo volumineux,Les bornes sont trop rapprochées pour passer f...,1710973981,0,0,3,0,Jacou
3,B9EAB99C,43.65346859600572,3.9190386235713954,"Chemin de Caylus, Castelnau-le-Lez",Barrières trop rapprochées passage carriole,Il y a 2 paires de barrière trop rapprochées a...,1710973443,0,0,3,0,Castelnau-le-Lez
4,6D94236F,43.6016298,3.883847,"Boulevard Rabelais, Montpellier","Peu de signalisation sur route refaite, danger",,1710939660,0,0,6,0,Montpellier
...,...,...,...,...,...,...,...,...,...,...,...,...
11255,89XQVRSN,43.60764910469699,3.87816989430601,"Place Molière, Montpellier",étalement dangereux de sable,risque d'accident de vélo et de piétons,1245427980,1,0,4,1,Montpellier
11256,GDECILXJ,43.59026055373006,3.8521778463105742,"Rue Jacques Fouroux, Montpellier",devant l'entrée d'un parc pour enfant,"gosses masqués par les voitures, poussettes gê...",1245257640,0,0,2,1,Montpellier
11257,534E0TBV,43.59244798819907,3.842795481584914,"Rue du Pas du Loup, Montpellier",squat permanent du trottoir,,1245255900,0,0,2,1,Montpellier
11258,Q6ZDHE60,43.59831843324888,3.8463419832739163,"Rue du Pas du Loup, Montpellier",pourquoi s'embêter à se garer correctement,,1245219600,0,0,2,1,Montpellier
